In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

23/02/25 10:14:43 WARN Utils: Your hostname, yujaeseong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.219.147 instead (on interface en0)
23/02/25 10:14:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/25 10:14:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/opt/homebrew/Caskroom/miniforge/base/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [11]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PUlocationID AS revenue_zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [12]:
df_green_revenue.show()

+-------------------+------------+------------------+--------------+
|               hour|revenue_zone|            amount|number_records|
+-------------------+------------+------------------+--------------+
|2020-01-01 00:00:00|           7| 769.7299999999996|            45|
|2020-01-01 00:00:00|          17|195.03000000000003|             9|
|2020-01-01 00:00:00|          18|               7.8|             1|
|2020-01-01 00:00:00|          22|              15.8|             1|
|2020-01-01 00:00:00|          24|              87.6|             3|
|2020-01-01 00:00:00|          25|             531.0|            26|
|2020-01-01 00:00:00|          29|              61.3|             1|
|2020-01-01 00:00:00|          32| 68.94999999999999|             2|
|2020-01-01 00:00:00|          33|            317.27|            11|
|2020-01-01 00:00:00|          35|            129.96|             5|
|2020-01-01 00:00:00|          36|            295.34|            11|
|2020-01-01 00:00:00|          37|

In [13]:
df_green_revenue.write.parquet('data/report/revenue/green')

23/02/25 10:20:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/02/25 10:20:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/02/25 10:20:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

/opt/homebrew/Caskroom/miniforge/base/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [15]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PUlocationID AS revenue_zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [16]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow')

23/02/25 10:29:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/02/25 10:29:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/02/25 10:29:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
